<a href="https://colab.research.google.com/github/kookeej/DILAB/blob/main/7.8-7.21/lab11-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab11-2 - RNN hihello and charseq
===
## 목차
1. hihello 문제
2. Data setting - one-hot encoding
3. Cross Entropy loss
4. Code run through

# 1. 'hihello' 문제
* 'h', 'i', 'h', 'e', 'l', 'l', 'o' 를 예측하기
* 'h'가 들어올 경우 'i' 또는 'e'가 될 가능성이 있다.
* 따라서 모델이 어디까지 진행되었는지를 저장하는 ```hidden state```의 역할이 중요하다.
* 파이토치에서 이 문자들을 어떻게 표현하는 것이 좋을까?

# 2. Data setting - one-hot encoding
파이토치에서 문자들을 나타내는 방법은 여러가지가 있다.
1. index
>각각에 characters에 index를 넣어준다. 하지만 숫자의 크기에 따라 의미를 갖게 되는 문제점이 있다.
2. one-hot encoding
>벡터의 원소 하나만 1로 활성화된다. 
![](https://mblogthumb-phinf.pstatic.net/MjAxOTA3MTJfMTA2/MDAxNTYyODYzMjgzOTk5.JO8akPyLBx4HDB27sprZtK_hwXrKtHVfg0gzhYsBRhIg.4r28fuL_3xBFusElkBPdZfQHBvsTcXCRC72l0j6FnlEg.PNG.kbsdr11/image.png?type=w800)    
여기서 ```x_one_hot```의 각 행은 각각 h, i, h, e, l, l를 나타낸다. 그런데 왜 'o'는 없을까? 이 모델은 'hihello'에서 다음 문자를 예측하도록 하는 모델이기 때문에 [h, i, h, e, l, l]까지는 input으로 사용되고 [i, h, e, l, l, o]은 output으로 사용된다.    
따라서 x_data를 살펴보면 'o'가 생략된 부분까지, y_data는 맨 앞의 'h'를 생략하고 끝까지 표현되었다.

# 3. Cross Entropy Loss
이번 예제에서는 ```cross entropy loss```를 사용한다. 이 손실함수는 ```categorical output```을 예측하는 모델에서 주로 사용된다. 먼저 softmax 함수를 통해 output을 0~1사이의 확률값으로 나타내준다. Cross entropy는 이 확률값을 정답 카테고리에 대해 최대한 증가시킨다. 
```
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
...
loss = criterion(outputs.view(-1, input_size), Y.view(-1))
```
여기서 주의할 점은 criterion의 첫 번째 파라미터에는 **이 모델의 output**을 주어야 하고, 두 번째 파라미터에는 **정답 레이블**을 주어야한다. 이 순서를 바꾸면 제대로 동작하지 않을 수 있다.     

# 4. Code run through
코드를 분석해본다.
## 4.1. hihello

In [ ]:
char_set = ['h', 'i', 'e', 'l', 'o']        # 먼저 character list에 대한 정의다.
#hyper parameters
input_size = len(char_set)                  # one-hot encoding이니 char_set의 개수만큼이다.
hidden_size = len(char_set)                 # hidden_size는 다른 값이어도 된다.
learning_rate = 0.1

#data setting
x_data = [[0, 1, 0, 2, 3, 3]]               # character들을 인덱스값으로 표현
x_one_hot = [[[1, 0, 0, 0, 0],              # 각 문자를 one-hot encoding으로 표현
              [0, 1, 0, 0, 0],
              [1, 0, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0],
              [0, 0, 0, 1, 0]]]
 y_data = [[1, 0, 2, 3, 3, 4]]

 # transform as torch tensor variable
 X = torch.FloatTensor(x_one_hot)           # 파이토치의 텐서로 변환해준다.
 Y = torch.LongTensor(y_data)

## 4.2. charseq
'hihello'와 달리 어떤 임의의 문자열이 sample값이 들어가도 동작할 수 있다. 

In [ ]:
sample = "if you want you"

# make dictionary
char_set = list(set(sample))        # 중복된 문자들은 제거하고 단일 문자들만 모아서 리스트로 만들어준다.
char_dic = {c: i for i, c in enumerate(char_set)}   # 특정 문자를 주면 그 인덱스로 찾아주는 딕셔너리
# enumerate를 쓰면 index와 char를 모두 가져올 수 있다.
# char to index로 mapping하여 딕셔너리로 만들어준다.


# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
learning_rate = 0.1

# data setting
sample_idx = [char_dic[c] for c in sample]      # char_dic을 이용해서 index들을 구해준다. 
x_data = [sample_idx[:-1]]                      # 맨 마지막 문자 제외
x_one_hot = [np.eye(dic_size)[x] for x in x_data]   # x_data의 인덱스로 one-hot vector를 만든다.
y_data = [sample_idx[1:]]                       # 맨 처음 문자 제외

# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

## 4.3. RNN을 만들고 학습하는 과정
여기서부터는 RNN을 만들고 학습하는 과정을 담은 코드이다.

In [ ]:
# declare RNN
rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # batch_first guarantees the order of output = (B, S, F)
                                                              # batch dimension이 가장 앞으로 온다.

# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

# start training
for i in range(100):
    optimizer.zero_grad()

    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

* 왜 batch dimension이 맨 앞으로 오도록 만드는걸까?

# 5. Full Code
## 5.1. Hihello

In [19]:
import torch
import torch.optim as optim
import numpy as np

In [20]:
# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

In [21]:
# declare dictionary
char_set = ['h', 'i', 'e', 'l', 'o']

In [22]:
# hyper parameters
input_size = len(char_set)
hidden_size = len(char_set)
learning_rate = 0.1

In [23]:
# data setting
x_data = [[0, 1, 0, 2, 3, 3]]
x_one_hot = [[[1, 0, 0, 0, 0],
              [0, 1, 0, 0, 0],
              [1, 0, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0],
              [0, 0, 0, 1, 0]]]
y_data = [[1, 0, 2, 3, 3, 4]]

In [24]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [25]:
# declare RNN
rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)  # batch_first guarantees the order of output = (B, S, F)

In [26]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

In [27]:
# start training
for i in range(100):
    optimizer.zero_grad()
    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.7802648544311523 prediction:  [[1 1 1 1 1 1]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  iiiiii
1 loss:  1.4931949377059937 prediction:  [[1 4 1 1 4 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ioiioo
2 loss:  1.3337111473083496 prediction:  [[1 3 2 3 1 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ilelio
3 loss:  1.2152944803237915 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  elelll
4 loss:  1.1131387948989868 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  elelll
5 loss:  1.0241864919662476 prediction:  [[2 3 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  elello
6 loss:  0.9573140740394592 prediction:  [[2 3 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  elello
7 loss:  0.9102001190185547 prediction:  [[2 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ehello
8 loss:  0.8731765747070312 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
9

## 5.2. charseq

In [28]:
import torch
import torch.optim as optim
import numpy as np

# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

In [29]:
sample = " if you want you"

In [30]:
# make dictionary
char_set = list(set(sample))
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_dic)

{'o': 0, ' ': 1, 'y': 2, 'w': 3, 't': 4, 'f': 5, 'a': 6, 'n': 7, 'u': 8, 'i': 9}


In [31]:
# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
learning_rate = 0.1

In [32]:
# data setting
sample_idx = [char_dic[c] for c in sample]
x_data = [sample_idx[:-1]]
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = [sample_idx[1:]]

In [33]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [34]:
# declare RNN
rnn = torch.nn.RNN(dic_size, hidden_size, batch_first=True)

In [35]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

In [37]:
# start training
for i in range(100):
    optimizer.zero_grad()
    outputs, _status = rnn(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  0.8685041666030884 prediction:  [[9 5 1 2 0 8 1 2 6 7 4 1 2 0 8]] true Y:  [[9, 5, 1, 2, 0, 8, 1, 3, 6, 7, 4, 1, 2, 0, 8]] prediction str:  if you yant you
1 loss:  0.868259608745575 prediction:  [[9 5 1 2 0 8 1 2 6 7 4 1 2 0 8]] true Y:  [[9, 5, 1, 2, 0, 8, 1, 3, 6, 7, 4, 1, 2, 0, 8]] prediction str:  if you yant you
2 loss:  0.8680338263511658 prediction:  [[9 5 1 2 0 8 1 2 6 7 4 1 2 0 8]] true Y:  [[9, 5, 1, 2, 0, 8, 1, 3, 6, 7, 4, 1, 2, 0, 8]] prediction str:  if you yant you
3 loss:  0.8678252696990967 prediction:  [[9 5 1 2 0 8 1 2 6 7 4 1 2 0 8]] true Y:  [[9, 5, 1, 2, 0, 8, 1, 3, 6, 7, 4, 1, 2, 0, 8]] prediction str:  if you yant you
4 loss:  0.8676319718360901 prediction:  [[9 5 1 2 0 8 1 2 6 7 4 1 2 0 8]] true Y:  [[9, 5, 1, 2, 0, 8, 1, 3, 6, 7, 4, 1, 2, 0, 8]] prediction str:  if you yant you
5 loss:  0.8674528002738953 prediction:  [[9 5 1 2 0 8 1 2 6 7 4 1 2 0 8]] true Y:  [[9, 5, 1, 2, 0, 8, 1, 3, 6, 7, 4, 1, 2, 0, 8]] prediction str:  if you yant you
6 loss:  0.

* ????????????????